In [1]:
import pandas as pd
import numpy as np
import imaplib
import email
import quopri
import re
import html
from bs4 import BeautifulSoup
import requests

In [2]:
username = 'prospectstudent@colby.edu'
password = 'Student.2017'

# username = 'haohaom2@gmail.com'
# password = '62947394'

### Accessing the email

In [3]:
mail = imaplib.IMAP4_SSL('imap.gmail.com')
mail.login(username, password)
mail.list()
# Out: list of "folders" aka labels in gmail.
mail.select("inbox") # connect to inbox.

('OK', [b'4340'])

In [4]:
result, data = mail.uid('search', None, "ALL") # search and return uids instead

first_email_id = data[0].split()[0]
latest_email_uid = data[0].split()[-1]

test_email_uid = data[0].split()[-4]

ids = data[0].split()

In [5]:
print('first email uid ', first_email_id)
print('latest_email_uid ', latest_email_uid)

first email uid  b'4009'
latest_email_uid  b'12577'


### Reading the email message

In [6]:
result, data = mail.uid('fetch', test_email_uid, '(RFC822)')
raw_email = str(data[0][1], 'utf-8')
email_message = email.message_from_string(raw_email)
print(email_message['subject'])
subject = email_message['subject']

Google Alert - "Jennifer Diamond"


In [7]:
def get_text_from_email(email_msg):
    if email_msg.is_multipart():
        s = ''
        for payload in email_msg.get_payload():
            s += payload.get_payload()
    else:
        s = b.get_payload()
    
    return s

In [8]:
# This cleans html stuff so I can extract the xml code
text = get_text_from_email(email_message).replace('\r\n', '').replace('=', '')

In [9]:
# text = html.unescape(text)
text

' News - 1 new result for ["Jennifer Diamond"] MILLBURY JR./SR. HIGH SCHOOL HONOR ROLLMillbury-Sutton ChronicleCrossman, Jennifer Diamond, Dylan Dwyer, Julia Fortin, Daniel Fuentes,Nicholas Lazzaro, Rowan Luff, Cole Mueller, John Prominski, ViviannaRodriguez, Emma Romeo, Victoria Rondeau, Jillian Salaway, Rigo Sanchez,Samantha Stafinski, Emma Taubert Grade 10. PRINCIPAL\'S LIST (Highest<https://www.google.com/url?rctj&sat&urlhttp://www.millburysutton.com/articles/millbury-jr-sr-high-school-honor-roll-4/&ctga&cdCAEYACoUMTU5NjUyMjMzOTU2MDE5Mzc5NTgyGjQxZWUzODg3MmI2ZDRmYmE6Y29tOmVuOlVT&usgAFQjCNFJUfO5HZt9uN0OEXWnBxlF2r4aaQ>- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -Unsubscribe from this Google Alert:<https://www.google.com/alerts/remove?sourcealertsmail&hlen&glUS&msgidMTU5NjUyMjMzOTU2MDE5Mzc5NTg&sAB2Xq4iVq7cu0xx3qrd0c27EYFq8DfiOx7tLgO4>Create another Google Alert:<https://www.google.com/alerts?sourcealertsmail&hlen&glUS&msgidMTU5NjUyMjMzOTU2MDE5Mzc5NTg>Sign in to ma

In [10]:
# extracts xml files from the raw string text
def extract_xml(raw_text):
    raw_text = raw_text.replace('\r\n', '').replace('=', '')
    m = re.findall('alerts/feeds\S+', raw_text)
    
    links = [('https://www.google.com/' + x).replace('\"', '') for x in m]
    return links

### Using Regex to get the links

In [11]:
# Passes in an xml link
# returns a list of links used in the website
def get_url_from_xml(xml_link):
    r  = requests.get(xml_link)
    soup = BeautifulSoup(r.content, 'html.parser')   # creates a BS4 from given xml link
    web_lists = soup.find_all('link')[1:]            # finds all the BS4 links
    
    urls = [re.findall('url=(.*?)&', url['href'])[0] for url in web_lists]
    return urls

In [12]:
get_url_from_xml(extract_xml(text)[0])

['http://www.millburysutton.com/articles/millbury-jr-sr-high-school-honor-roll-4/']

In [13]:
url = get_url_from_xml(extract_xml(text)[0])[0]
url

'http://www.millburysutton.com/articles/millbury-jr-sr-high-school-honor-roll-4/'

### Scrapes each link and creates a keyward card

In [14]:
r  = requests.get(url)

soup = BeautifulSoup(r.content, 'html.parser')

In [15]:

for paragraph in soup.find_all('p'):
    print(paragraph.text)

News for Millbury and Sutton, Massachusetts
Thursday, May 3, 2018


By 

        Millbury-Sutton Chronicle    
| 
 on 
        May 03, 2018    
Grade 12 
PRINCIPAL’S LIST (Highest 
Honors) 
Ashley Briddon, Enzo,
Croghan, Juliana Dick,
Thomas Machacz, Katherine
O’Leary, Madison Plante,
Emma Turner, Madison
Wilson
HONORS 
Brenna Berube, Hannah
Bishop, Samantha Brady,
Sandra Carrignan, Makayla
Cesaitis, Cassidy Clark,
Chase Carrion, Sydney
Corridori, Lyndsey Davolio,
Olivia Dean, Zoey Dean,
Vincent DiDomenica, Joseph
Favreau, Elijah Friedman,
Abigail Goyette, Amanda
Larson, Paige Mathie, Ethan
McDuffie, Jenna Nasuti,
Taylor Palmer, Merideth
Pine, James Saunders, Alexis
Violette, Kayley Watkins,
Devin Weaver
Grade 11 
PRINCIPAL’S LIST (Highest 
Honors) 
Allison Drapeau, Timothy
Dunne, Samantha Fullen,
Noah Hopkins, Sydney
Moore, Brianna Poske, Kyle
Taubert
HONORS 
Emily Beaumier, Aradhana
Bissoondial, Harrison
Bourdeau, Jaresa Burgos,
Trenton Cano, Sarah
Crossman, Jennifer Diamond,
Dylan D

In [16]:
[paragraph.text.split() for paragraph in soup.find_all('p')]

[['News', 'for', 'Millbury', 'and', 'Sutton,', 'Massachusetts'],
 ['Thursday,', 'May', '3,', '2018'],
 [],
 ['By', 'Millbury-Sutton', 'Chronicle', '|', 'on', 'May', '03,', '2018'],
 ['Grade',
  '12',
  'PRINCIPAL’S',
  'LIST',
  '(Highest',
  'Honors)',
  'Ashley',
  'Briddon,',
  'Enzo,',
  'Croghan,',
  'Juliana',
  'Dick,',
  'Thomas',
  'Machacz,',
  'Katherine',
  'O’Leary,',
  'Madison',
  'Plante,',
  'Emma',
  'Turner,',
  'Madison',
  'Wilson',
  'HONORS',
  'Brenna',
  'Berube,',
  'Hannah',
  'Bishop,',
  'Samantha',
  'Brady,',
  'Sandra',
  'Carrignan,',
  'Makayla',
  'Cesaitis,',
  'Cassidy',
  'Clark,',
  'Chase',
  'Carrion,',
  'Sydney',
  'Corridori,',
  'Lyndsey',
  'Davolio,',
  'Olivia',
  'Dean,',
  'Zoey',
  'Dean,',
  'Vincent',
  'DiDomenica,',
  'Joseph',
  'Favreau,',
  'Elijah',
  'Friedman,',
  'Abigail',
  'Goyette,',
  'Amanda',
  'Larson,',
  'Paige',
  'Mathie,',
  'Ethan',
  'McDuffie,',
  'Jenna',
  'Nasuti,',
  'Taylor',
  'Palmer,',
  'Merideth',
 

### Clean text of punctuations and store them all in a list 

In [17]:
words = []
for paragraph in soup.find_all('p'):
    words += paragraph.text.split()

In [18]:
words

['News',
 'for',
 'Millbury',
 'and',
 'Sutton,',
 'Massachusetts',
 'Thursday,',
 'May',
 '3,',
 '2018',
 'By',
 'Millbury-Sutton',
 'Chronicle',
 '|',
 'on',
 'May',
 '03,',
 '2018',
 'Grade',
 '12',
 'PRINCIPAL’S',
 'LIST',
 '(Highest',
 'Honors)',
 'Ashley',
 'Briddon,',
 'Enzo,',
 'Croghan,',
 'Juliana',
 'Dick,',
 'Thomas',
 'Machacz,',
 'Katherine',
 'O’Leary,',
 'Madison',
 'Plante,',
 'Emma',
 'Turner,',
 'Madison',
 'Wilson',
 'HONORS',
 'Brenna',
 'Berube,',
 'Hannah',
 'Bishop,',
 'Samantha',
 'Brady,',
 'Sandra',
 'Carrignan,',
 'Makayla',
 'Cesaitis,',
 'Cassidy',
 'Clark,',
 'Chase',
 'Carrion,',
 'Sydney',
 'Corridori,',
 'Lyndsey',
 'Davolio,',
 'Olivia',
 'Dean,',
 'Zoey',
 'Dean,',
 'Vincent',
 'DiDomenica,',
 'Joseph',
 'Favreau,',
 'Elijah',
 'Friedman,',
 'Abigail',
 'Goyette,',
 'Amanda',
 'Larson,',
 'Paige',
 'Mathie,',
 'Ethan',
 'McDuffie,',
 'Jenna',
 'Nasuti,',
 'Taylor',
 'Palmer,',
 'Merideth',
 'Pine,',
 'James',
 'Saunders,',
 'Alexis',
 'Violette,',
 '

In [19]:
# Saves words as a list of all the words that appear
import string
words = list(map(lambda x: x.strip(string.punctuation), words))

### Retrieve Subject information from database

In [37]:
name = re.findall('"(.*?)"', subject)[0]
name

'Jennifer Diamond'

### Uses count vectorizer to get a count of key words

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
vec = CountVectorizer()
keys = ['Amy', 'Walter', 'Colby', 'Cook', 'Political']
X = vec.fit_transform(keys)

In [22]:
sum(vec.transform(words).toarray(), 1) - 1

array([0, 1, 0, 0, 0])

In [23]:
'Endomag' in words

False

In [30]:

m = re.findall('alerts/feeds\S+', text)
url = ('https://www.google.com/' + m[0]).replace('\"', '')
url

'https://www.google.com/alerts/feeds/16218955178302834825/8278055857187349425'

In [31]:
r  = requests.get(url)

soup = BeautifulSoup(r.content, 'html.parser')

In [32]:
# use &ct as a stopper

web_lists = soup.find_all('link')[1:]
for web in web_lists:
    print(web['href'])

https://www.google.com/url?rct=j&sa=t&url=http://www.millburysutton.com/articles/millbury-jr-sr-high-school-honor-roll-4/&ct=ga&cd=CAIyGjQxZWUzODg3MmI2ZDRmYmE6Y29tOmVuOlVT&usg=AFQjCNFJUfO5HZt9uN0OEXWnBxlF2r4aaQ


In [33]:
url = web_lists[0]['href']
url

'https://www.google.com/url?rct=j&sa=t&url=http://www.millburysutton.com/articles/millbury-jr-sr-high-school-honor-roll-4/&ct=ga&cd=CAIyGjQxZWUzODg3MmI2ZDRmYmE6Y29tOmVuOlVT&usg=AFQjCNFJUfO5HZt9uN0OEXWnBxlF2r4aaQ'

In [34]:
[re.findall('url=(.*?)&', url['href'])[0] for url in web_lists]

['http://www.millburysutton.com/articles/millbury-jr-sr-high-school-honor-roll-4/']

In [110]:
test_soup = BeautifulSoup(requests.get(url).content, 'html.parser')

In [111]:
test_soup.get_text()

'window.googleJavaScriptRedirect=1var n={navigateTo:function(b,a,d){if(b!=a&&b.google){if(b.google.r){b.google.r=0;b.location.href=d;a.location.replace("about:blank");}}else{a.location.replace(d);}}};n.navigateTo(window.parent,window,"http://www.nydailynews.com/new-york/woman-gave-no-hints-trouble-found-butchered-park-article-1.3955035");\n'

In [89]:
def get_first_text_block(email_message_instance):
    maintype = email_message_instance.get_content_maintype()
    if maintype == 'multipart':
        for part in email_message_instance.get_payload():
            if part.get_content_maintype() == 'text':
                return part.get_payload()
    elif maintype == 'text':
        return email_message_instance.get_payload()

In [107]:
# get_first_text_block(email_message)

#### using mail.search()

In [70]:
type, data = mail.search(None, 'ALL')
mail_ids = data[0].decode()
id_list = mail_ids.split()

In [71]:
first_email_id = int(id_list[0])
latest_email_id = int(id_list[-1])

In [89]:
typ, data = mail.fetch('4', '(RFC822)' )

In [91]:
for response_part in data:
    if isinstance(response_part, tuple):
        msg = email.message_from_string(str(response_part[1]))
        email_subject = msg['subject']
        email_from = msg['from']
        print( 'From : ' + str(email_from) + '\n')
        print ('Subject : ' + str(email_subject) + '\n')

From : None

Subject : None



In [97]:
raw_email = data[0][1]
email_message = email.message_from_string(str(raw_email))
print(email_message['To'])

None


In [84]:
for response_part in data:
    if isinstance(response_part, tuple):
        print(type(response_part[1]))

<class 'bytes'>


In [65]:
import smtplib
import time
import imaplib
import email
def read_email_from_gmail(user, pw):
    try:
        mail = imaplib.IMAP4_SSL(SMTP_SERVER)
        mail.login(user, pw)
        mail.select('inbox')

        typ, data = mail.search(None, 'ALL')
        mail_ids = data[0]

        id_list = mail_ids.split()   
        first_email_id = int(id_list[0])
        latest_email_id = int(id_list[-1])


        for i in range(latest_email_id,first_email_id, -1):
            typ, data = mail.fetch(str(i), '(RFC822)' )

            for response_part in data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_string(response_part[1])
                    email_subject = msg['subject']
                    email_from = msg['from']
                    print( 'From : ' + email_from + '\n')
                    print ('Subject : ' + email_subject + '\n')

    except Exception as e:
        print (e)

In [66]:
read_email_from_gmail(username, password)

initial_value must be str or None, not bytes
